List of modules

In [4]:
import json

import pickle
import numpy as np

from googleapiclient.discovery import build

from operator import itemgetter

import os
from os import listdir
from os.path import isfile, join
import string
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [5]:
def readfile():
    with open('ssp2.json', 'r') as fp: 
        complistnew = json.load(fp)

    cmlistnew = sorted(complistnew, key=itemgetter('Video'))
    
    return cmlistnew

In [6]:
def readdata():
    
    datapath = os.getcwd() +"\\yourpath\\trainfile"
    onlyfiles = [f for f in listdir(datapath) if isfile(join(datapath, f))]

    return onlyfiles

In [7]:
def determineclass(persvalue, i):
    if persvalue >= 0.5:
        if i == 0:
            newlabel = 'High Extraversion'
        else:
            newlabel = 'High Neuroticism'
    else:
        if i == 0:
            newlabel = 'Low Extraversion'
        else:
            newlabel = 'Low Neuroticism'
    
    return newlabel

Extraction code

#DATA PREPROCESSING

#filter comments on appearance

In [320]:
appearance_wordlist =[]

with open('appwords.txt','r') as f:
    for line in f:
        for word in line.split():
            appearance_wordlist.append(word)
    

print(appearance_wordlist)

['appearance', 'adorable', 'adventurous', 'aggressive', 'alert', 'attractive', 'average', 'beautiful', 'blue-eyed', 'bloody', 'blushing', 'bright', 'clean', 'clear', 'cloudy', 'colorful', 'crowded', 'cute', 'dark', 'drab', 'distinct', 'dull', 'elegant', 'excited', 'fancy', 'filthy', 'glamorous', 'gleaming', 'gorgeous', 'graceful', 'grotesque', 'handsome', 'homely', 'light', 'long', 'magnificent', 'misty', 'motionless', 'muddy', 'old-fashioned', 'plain', 'poised', 'precious', 'quaint', 'shiny', 'smoggy', 'sparkling', 'spotless', 'stromy', 'strange', 'ugly', 'ugliest', 'unsightly', 'unusual', 'wide-eyed', 'condition', 'alive', 'annoying', 'bad', 'better', 'beautiful', 'brainy', 'breakable', 'busy', 'careful', 'clever', 'crazy', 'dead', 'easy', 'expensive', 'fragile', 'frail', 'helpful', 'gifted', 'modern', 'open', 'real', 'super', 'feeling', 'angry', 'arrogant', 'awful', 'black', 'bored', 'confused', 'cruel', 'dizzy', 'evil', 'grumpy', 'lazy', 'naughty', 'nutty', 'terrible', 'tired', 'up

In [324]:
complistnew = readfile()
videoexample=1

commentexample=complistnew[videoexample]['Data']['Comment']

# print("length before video 10: ", len(complistnew[videoexample]['Data']['Comment']), " \n")

# print("Comment out the list, before filter :  ' ", commentexample, "'  \n")
list_number_comments_b4=[]
list_number_comments_after=[]

for i, video in enumerate(complistnew):
    video_comments= video['Data']['Comment']
    indices= []
    for j, comment in enumerate(video_comments):
        comment2= comment.split()
        if any(item in appearance_wordlist for item in comment2 )==True:
#             print("didn't pop:" , comment)
            indices.append(j)
    video['Data']['Comment']= [video_comments[k] for k in indices]
    list_number_comments_after.append(len(video['Data']['Comment']))


print("average amount comments after filter= ", np.mean(list_number_comments_after))



# commentexample=complistnew[videoexample]['Data']['Comment']

# print("length after video 10: ", len(complistnew[videoexample]['Data']['Comment']), " \n")

# print("Comment out the list, containing 'love' : ", commentexample)


average amount comments=  15.096491228070175


# DECAPITALIZE WORDS

In [37]:

#   import nltk
# from nltk.stem.porter import PorterStemmer

# from nltk.tokenize import word_tokenize
# # nltk.download('punkt')

In [38]:
#Stem the words (no capitals)
# porter = PorterStemmer()
# stems = []

# for i, video in enumerate(complistnew):
#     for j, comment in enumerate(video['Data']['Comment']):
#         tokens = word_tokenize(complistnew[0]['Data']['Comment'][0])
#         for t in tokens:
#             stem.append(porter.stem(t))

## example
# for t in comment:    
#     stems.append(porter.stem(t))
# print(stems)


Data initialization

In [9]:
comment_data_pos = []
label_data_pos = []
comment_data_neg = []
label_data_neg = []

category = ['Extraversion', 'Neuroticism']

Create extraversion data

In [10]:
count_pos = 0
count_neg = 0
for viddata in complistnew:
    tempcom = ''
    templist = []
    extr = determineclass(viddata['Personality']['Extraversion'], 0)
    if extr == 'High Extraversion':
        for comdata in viddata['Data']['Comment']:
            if tempcom == '':
                tempcom = comdata
            else:
                tempcom = tempcom + " " + comdata
        comment_data_pos.append(tempcom)
        label_data_pos.append('TEST_POS_%s' % count_pos)
        count_pos = count_pos + 1
    else:
        for comdata in viddata['Data']['Comment']:
            if tempcom == '':
                tempcom = comdata
            else:
                tempcom = tempcom + " " + comdata
        comment_data_neg.append(tempcom)
        label_data_neg.append('TEST_NEG_%s' % count_neg)
        count_neg = count_neg + 1

Build gensim model

In [11]:
docs = []
for i in range(len(comment_data_pos)): 
    comment_data_pos[i] = comment_data_pos[i].lower()
    comment_data_pos[i] = comment_data_pos[i].translate(str.maketrans('', '', string.punctuation))
    
    docs.append(TaggedDocument(words=comment_data_pos[i].split(), tags=[label_data_pos[i]]))

for i in range(len(comment_data_neg)): 
    comment_data_neg[i] = comment_data_neg[i].lower()
    comment_data_neg[i] = comment_data_neg[i].translate(str.maketrans('', '', string.punctuation))
    
    docs.append(TaggedDocument(words=comment_data_neg[i].split(), tags=[label_data_neg[i]]))
        
model = Doc2Vec(min_count=1, window=10, vector_size=100, sample=1e-4, negative=5, workers=7)

model.build_vocab(docs)

shuffled = list(docs)
random.shuffle(shuffled)

for epoch in range(10):
    model.train(shuffled, total_examples=model.corpus_count, epochs=model.epochs)
    
model.save(os.path.abspath('') + '\\extraversion_model.d2v')

KeyboardInterrupt: 

Assigning model to train and test data

In [9]:
model_train = Doc2Vec.load(os.path.abspath('') + '\\baseline_model.d2v')
model_test = Doc2Vec.load(os.path.abspath('') + '\\extraversion_model.d2v')

train_arrays = np.zeros((25000, 100))
train_labels = np.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model_train[prefix_train_pos]
    train_arrays[12500 + i] = model_train[prefix_train_neg]
    train_labels[i] = 0
    train_labels[12500 + i] = 1

test_arrays = np.zeros((2667, 100))
test_labels = np.zeros(2667)

for i in range(len(comment_data_pos)):
    prefix_test_pos = 'TEST_POS_' + str(i)
    test_arrays[i] = model_test[prefix_test_pos]
    test_labels[i] = 0

cc = len(comment_data_pos)

for i in range(len(comment_data_neg)):
    prefix_test_neg = 'TEST_NEG_' + str(i+cc)
    test_arrays[i+cc] = model_test[prefix_test_neg]
    test_labels[i+cc] = 1

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Robertie\\Documents\\JupyterLab\\SSPPersonalityPredictor\\baseline_model.d2v.trainables.syn1neg.npy'

Classify

In [ ]:
classifier = linear_model.LogisticRegression()
classifier.fit(train_arrays, train_labels)

pred = classifier.predict(test_arrays)

print(classifier.score(test_arrays, test_labels))
print(classification_report(test_labels, pred, digits = 3))

# BELOW TEMPORARY UNUSED

Pipeline Count

In [ ]:
NB_pipeline_cnt = Pipeline([
                ('count', CountVectorizer(lowercase=True, ngram_range=(1,2), stop_words='english')),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
                        ])

Check Performance Count

In [ ]:
count = 0
for cat in category:
    print('... Processing {}'.format(cat))
    NB_pipeline_cnt.fit(comment_data_train, label_data_train[count])
    # compute the testing accuracy
    prediction = NB_pipeline_cnt.predict(comment_data_test)
    print('Test accuracy is {}'.format(accuracy_score(label_data_test[count], prediction)))

Pipeline TfIdf

In [ ]:
NB_pipeline_tid = Pipeline([
                ('count', TfIdfVectorizer(lowercase=True, ngram_range=(1,2), stop_words='english')),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
                        ])

Check Performance TfIdf

In [ ]:
count = 0
for cat in category:
    print('... Processing {}'.format(cat))
    NB_pipeline_tid.fit(comment_data_train, label_data_train[count])
    # compute the testing accuracy
    prediction = NB_pipeline_tid.predict(comment_data_test)
    print('Test accuracy is {}'.format(accuracy_score(label_data_test[count], prediction)))